In [1]:
!pip install kaggle

Found existing installation: kaggle 1.5.6
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
  Using cached kaggle-1.5.6-py3-none-any.whl


In [2]:
! conda install -c conda-forge librosa --yes

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.4
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs:
    - librosa


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    astroid-2.5                |   py36h5fab9bb_1         297 KB  conda-forge
    flask-cors-3.0.8           |             py_0          14 KB  conda-forge
    fsspec-0.8.7               |     pyhd8ed1ab_0          72 KB  conda-forge
    lxml-4.6.2                 |   py36h04a5ba7_1         1.5 MB  conda-forge
    pillow-8.1.0               |   py36ha6010c0_2         670 KB  conda-forge
    pip-21.0.1                 |     pyhd8ed1ab_

In [6]:
import sagemaker
import boto3
from sagemaker import get_execution_role


import pandas as pd
import librosa
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from sklearn.utils import shuffle
import json
from collections import Counter
from tqdm import tqdm
import os
from scipy import signal

In [3]:
sagemaker_session = sagemaker.Session()
sagemaker_session.boto_region_name
role = get_execution_role()

bucket = 'bird-sound-data'
s3 = boto3.resource('s3') 

Download data from Kaggle

In [7]:
kaggle_api = {"username":"<Kaggle Name >","key":"<API key>"}

with open('kaggle.json','w') as f:
    f.write(json.dumps(kaggle_api))
    
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/home/ec2-user/.kaggle’: File exists


In [26]:
!kaggle competitions download -c birdclef-2021 -f train_metadata.csv
!unzip train_metadata.csv.zip
!rm train_metadata.csv.zip

  0%|                                               | 0.00/1.94M [00:00<?, ?B/s]
100%|██████████████████████████████████████| 1.94M/1.94M [00:00<00:00, 92.3MB/s]
Archive:  train_metadata.csv.zip
replace train_metadata.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


Filter the dataset using the metadata

In [8]:
meta_data_path = 'train_metadata.csv'

meta_data = pd.read_csv(meta_data_path)

meta_data = meta_data[(meta_data['latitude']>35)&(meta_data['latitude']<70)&(meta_data['longitude']>-10)&(meta_data['longitude']<60)]

counter_classes = pd.DataFrame(sorted(Counter(meta_data['primary_label']).items(), key=lambda x: x[1], reverse=True))
meta_data = meta_data.merge(counter_classes, how='left', left_on='primary_label', right_on=0)

del meta_data[0]
meta_data = meta_data.rename(columns={1:'count_class'})

meta_data = meta_data[meta_data['count_class']>= 100]
meta_data = shuffle(meta_data)
audio_name = meta_data['filename'].values
audio_labels = meta_data['primary_label'].values

cls_id_map = {name:idx for idx, name in enumerate(np.unique(audio_labels))}

Function used to compute the STFT

In [9]:
SR = 22050
SIGNAL_LENGTH = 5 
DURATION  = 10

def open_audio(path):
    waves, rate = librosa.load(path, sr=SR, duration=DURATION)
    processed_audios = []
    for i in range(0, len(waves), int(SIGNAL_LENGTH * SR)):
        w = waves[i:i + int(SIGNAL_LENGTH * SR)]
        if len(w) < int(SIGNAL_LENGTH * SR):
            break
        if np.any(np.isnan(w)) == False:
            stft = signal.stft(w, nperseg=SR*5//256)[-1]
            stft = np.abs(stft + 1e-10)
            stft = np.log(stft)
            stft = stft - np.min(stft)
            stft = stft/(np.max(stft) + 1e-10)
            processed_audios.append(stft)
    return processed_audios

Main function that downloads the audio and transfers it into the S3 bucket.

In [ ]:
!mkdir audio_files

dir_path = 'audio_files/'
data_path = 'data/'

for audio_file, target in tqdm(zip(audio_name[:4000], audio_labels[:4000])):
    !kaggle competitions download -c birdclef-2021 -p audio_files/ -f {'train_short_audio/' + target + '/' + audio_file} 
    audio_path = dir_path + str(audio_file)
    try:
        temp = open_audio(audio_path)
        if not os.path.exists(data_path + target):
            os.makedirs(data_path + target)

        for idx, x in enumerate(temp):
            img = Image.fromarray(x * 255.0).convert("L")
            img_save_path = data_path + target + '/' + audio_file[:-4] + f'_{idx}' '.png'
            img.save(img_save_path)
            s3.Bucket(bucket).upload_file(img_save_path, img_save_path)
            !rm {img_save_path}
        !rm {dir_path + audio_file}

    except:
        continue

mkdir: cannot create directory ‘audio_files’: File exists



0it [00:00, ?it/s]


  0%|                                                | 0.00/250k [00:00<?, ?B/s]

100%|████████████████████████████████████████| 250k/250k [00:00<00:00, 40.6MB/s]



1it [00:02,  2.79s/it]

  0%|                                                | 0.00/179k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 179k/179k [00:00<00:00, 37.3MB/s]


...


103it [03:02,  1.90s/it]

429 - Too Many Requests


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")

104it [04:33, 28.60s/it]

429 - Too Many Requests


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")

105it [06:03, 47.29s/it]